In [ ]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import numpy as np
import cv2
import os
import albumentations as A
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import random

# 1. Get Train Labels csv

In [3]:
df = pd.read_csv('../Labels/AllLabels.csv')
df.head()
print(df.shape)

(8925, 5)


In [4]:
# Cargar Labels del CSV
def load_labels(csv_path):
    df = pd.read_csv(csv_path)
    labels_dict = {}
    for _, row in df.iterrows():
        clip_id = row['ClipID'].replace('.avi', '').replace('.mp4', '')
        labels_dict[clip_id] = [row['Boredom'], row['Engagement'], row['Confusion'], row['Frustration']]
    return labels_dict

In [5]:
# Procesar dataset
def process_dataset(frames_root_folder):
    frames_root_folder = Path(frames_root_folder)
    video_folders = list(frames_root_folder.glob("*/*"))
    with open("video_folders.txt", "w") as f:
        for folder in video_folders:
            folder = str(folder).split("\\")[-1]
            f.write(str(folder) + "\n")

In [9]:
frames_root = "output_frames/Train"
process_dataset(frames_root)

In [10]:
# Load the text file with ClipIDs
with open("video_folders.txt", "r") as f:
    clip_ids = [line.strip() for line in f]
df['ClipID'] = df['ClipID'].str.replace('.avi', '').str.replace('.mp4', '')
filtered_df = df[df['ClipID'].isin(clip_ids)]
filtered_df.shape

(5481, 5)

In [ ]:
# save filtered_df to a csv file
filtered_df.to_csv("../Labels/TrainLabels.csv", index=False)

# 2. Get Frames per Video

In [23]:
def extract_frames_from_video(video_path, output_folder, max_frames=None):
    cap = cv2.VideoCapture(str(video_path))
    frame_count = 0
    success, frame = cap.read()
    
    while success:
        if max_frames and frame_count >= max_frames:
            break
        frame_filename = output_folder / f"frame_{frame_count:04d}.jpg"
        cv2.imwrite(str(frame_filename), frame)
        success, frame = cap.read()
        frame_count += 1
    cap.release()

def get_all_video_paths(dataset_dir, subset="Train"):
    dataset_path = Path(dataset_dir) / subset
    video_paths = []
    for person_folder in dataset_path.iterdir():
        if person_folder.is_dir():
            for video_folder in person_folder.iterdir():
                if video_folder.is_dir():
                    for video_file in video_folder.glob("*.*"):
                        if video_file.suffix.lower() in [".avi", ".mp4"]:
                            video_paths.append(video_file)

    return video_paths

def getFramesPerVideo(dataset_dir, subset="Train",max_frames_per_video=None, output_base="output_frames"):
    video_paths = get_all_video_paths(dataset_dir, subset=subset)
    print(f"Procesando {len(video_paths)} videos del conjunto {subset}...")

    for video_path in tqdm(video_paths, desc="Extrayendo frames"):
        relative_path = video_path.relative_to(dataset_dir)
        output_folder = Path(output_base) / relative_path.parent
        output_folder.mkdir(parents=True, exist_ok=True)
        extract_frames_from_video(video_path, output_folder, max_frames=max_frames_per_video)

In [24]:
subset = "Train"  # Cambia a "Test" o "Validation" según sea necesario
getFramesPerVideo(
    dataset_dir="../Datasets/DaiSee/DAiSEE/DataSet/",          # Ruta raíz al dataset DAiSEE
    subset=subset,
    #max_videos=6000,             # Cuántos videos procesar
    max_frames_per_video=75,       # Frames máximos por video (None = todos)
    output_base="output_frames"   # Carpeta donde guardar los frames extraídos
)

Procesando 5481 videos del conjunto Train...


Extrayendo frames: 100%|██████████| 5481/5481 [1:03:58<00:00,  1.43it/s]


# 3. Only Engagement Level

In [26]:
df_train = pd.read_csv("../Labels/TrainLabels.csv")
print(df_train.head())
print(df_train.shape)

       ClipID  Boredom  Engagement  Confusion  Frustration
0  1100011002        0           2          0            0
1  1100011003        0           2          0            0
2  1100011004        0           3          0            0
3  1100011005        0           3          0            0
4  1100011006        0           3          0            0
(5481, 5)


In [27]:
# Filtra los ClipIDs con engagement == 0
df_train_filtered = df_train.drop(columns=["Boredom", "Confusion", "Frustration"])
df_train_filtered['Engagement'] = df_train_filtered['Engagement'].replace({0: 0, 1: 0, 2: 1, 3: 1})
df_train_filtered = df_train_filtered[df_train_filtered['Engagement'] == 0]

# Set con los ClipID válidos
valid_clip_ids = set(df_train_filtered['ClipID'].astype(str))
valid_clip_ids

{'1100021003',
 '1100021055',
 '1100022005',
 '1100042023',
 '1100042026',
 '1100051007',
 '1100051016',
 '1100051030',
 '1100051031',
 '1100051053',
 '1100052014',
 '1100062008',
 '1100062045',
 '1100062049',
 '1100112002',
 '1100112006',
 '1100122056',
 '1100131017',
 '1100141013',
 '1100141027',
 '1100142033',
 '1100151011',
 '1100151057',
 '1100152010',
 '1100152017',
 '1100152031',
 '1100152055',
 '1100152070',
 '1100161053',
 '1100162005',
 '1100162016',
 '1100171004',
 '1100171008',
 '1100171059',
 '1100172012',
 '1100172017',
 '1100172033',
 '1100172034',
 '1100172043',
 '1100172058',
 '1100412018',
 '1100412033',
 '1100412039',
 '1110031010',
 '1110031025',
 '1110031027',
 '1110031033',
 '1110031038',
 '1110031056',
 '1110031063',
 '1110032014',
 '1110032027',
 '1110032043',
 '1813740138',
 '1813740184',
 '1813740185',
 '2000491077',
 '2000501006',
 '2000501030',
 '2000502053',
 '2000502065',
 '2000502081',
 '2026140257',
 '2026140264',
 '2026140273',
 '2056010134',
 '20560102

In [32]:
len(valid_clip_ids)

248

In [28]:
from pathlib import Path
import cv2
from tqdm import tqdm

def extract_frames_from_video(video_path, output_folder, max_frames=None):
    cap = cv2.VideoCapture(str(video_path))
    frame_count = 0
    success, frame = cap.read()
    
    while success:
        if max_frames and frame_count >= max_frames:
            break
        frame_filename = output_folder / f"frame_{frame_count:04d}.jpg"
        cv2.imwrite(str(frame_filename), frame)
        success, frame = cap.read()
        frame_count += 1
    cap.release()

def get_all_video_paths(dataset_dir, subset="Train"):
    dataset_path = Path(dataset_dir) / subset
    video_paths = []
    for person_folder in dataset_path.iterdir():
        if person_folder.is_dir():
            for video_folder in person_folder.iterdir():
                if video_folder.is_dir():
                    for video_file in video_folder.glob("*.*"):
                        if video_file.suffix.lower() in [".avi", ".mp4"]:
                            video_paths.append(video_file)
    return video_paths

def getFramesPerVideo_filtered(dataset_dir, valid_clip_ids, subset="Train", max_frames_per_video=None, output_base="Train_Augmentation"):
    video_paths = get_all_video_paths(dataset_dir, subset=subset)
    print(f"Procesando {len(video_paths)} videos del conjunto {subset}...")

    for video_path in tqdm(video_paths, desc="Extrayendo frames"):
        relative_path = video_path.relative_to(dataset_dir)
        person_id = relative_path.parts[1]  # e.g. Train/person123
        clip_id = relative_path.parts[2]    # clip_id (última carpeta antes del video)

        if clip_id in valid_clip_ids:
            # Genera el nuevo path: Train_Augmentation/person_id/clip_id/frames
            output_folder = Path(output_base) / person_id / clip_id
            output_folder.mkdir(parents=True, exist_ok=True)
            extract_frames_from_video(video_path, output_folder, max_frames=max_frames_per_video)

In [31]:
getFramesPerVideo_filtered(
    dataset_dir="../../Datasets/DaiSee/DAiSEE/DataSet/",
    valid_clip_ids=valid_clip_ids,
    subset="Train",
    max_frames_per_video=5,  # o el número que necesites
    output_base="../output_frames/Train_Augmentation"
)

Procesando 5481 videos del conjunto Train...


Extrayendo frames: 100%|██████████| 5481/5481 [00:05<00:00, 1043.77it/s]


## Data Augmentation

In [ ]:
# Define las transformaciones
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.GaussianBlur(p=0.3),
    A.HueSaturationValue(p=0.3),
    A.RandomScale(scale_limit=0.1, p=0.5),
    A.GaussNoise(p=0.3),
])

def augment_and_save_images(input_dir, output_dir, augment_times=5):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    for img_name in os.listdir(input_dir):
        if img_name.lower().endswith(".jpg"):
            img_path = input_dir / img_name
            image = cv2.imread(str(img_path))

            for i in range(augment_times):
                augmented = augmentations(image=image)["image"]
                aug_filename = output_dir / f"{img_name[:-4]}_aug{i}.jpg"
                cv2.imwrite(str(aug_filename), augmented)


In [ ]:
from deepface import DeepFace
def load_labels(csv_path):
    df = pd.read_csv(csv_path)
    labels_dict = {}
    for _, row in df.iterrows():
        # Convertir ClipID a str antes de usar replace y luego a int
        clip_id = str(row['ClipID']).replace('.avi', '').replace('.mp4', '')
        labels_dict[clip_id] = [
            int(row['Boredom']),
            int(row['Engagement']),
            int(row['Confusion']),
            int(row['Frustration'])
        ]
    return labels_dict

In [34]:
def extract_facenet_embedding(img_path):
    try:
        embedding = DeepFace.represent(img_path=img_path, model_name='Facenet', enforce_detection=False)
        return np.array(embedding[0]['embedding'])
    except Exception as e:
        print(f"Error en {img_path}: {e}")
        return None

In [35]:
def process_dataset(frames_root_folder, labels_csv, max_videos=1250):
    labels = load_labels(labels_csv)
    X = []
    y = []

    # Obtener todas las carpetas de video
    frames_root_folder = Path(frames_root_folder)
    video_folders = list(frames_root_folder.glob("*/*"))
    print(f"Total de carpetas de video encontradas: {len(video_folders)}")

    # Limitar a los primeros `max_videos`
    video_folders = video_folders[:max_videos]
    print(f"Procesando {len(video_folders)} carpetas de video...")

    # Procesar cada carpeta de video
    for video_folder in tqdm(video_folders, desc="Procesando videos"):
        clip_id = video_folder.name

        # Verificar si el clip_id está en labels
        if clip_id not in labels:
            with open("missing_clip_ids.txt", "a") as f:
                f.write(clip_id + "\n")
            print(f"ClipID {clip_id} no encontrado en labels.")
            continue

        emotion_levels = labels[clip_id]  # lista de 4 números

        # Crear un vector one-hot de 16 posiciones
        label_vector = np.zeros(16)
        for i, level in enumerate(emotion_levels):  # i = 0 (boredom), 1 (engagement), 2 (confusion), 3 (frustration)
            index = i * 4 + level  # cada emoción tiene 4 niveles
            label_vector[index] = 1

        video_embeddings = []
        
        # Procesar los frames dentro de esta carpeta
        for frame_path in video_folder.glob("*.jpg"):
            embedding = extract_facenet_embedding(str(frame_path))
            if embedding is not None:
                video_embeddings.append(embedding)
        
        video_embedding = np.mean(video_embeddings, axis=0)
        video_embedding = np.array(video_embedding)
        
        X.append(video_embedding)
        y.append(label_vector)

    X = np.array(X)
    y = np.array(y)

    print(f"Dataset procesado: {X.shape[0]} ejemplos.")
    return X, y

In [41]:
# Procesar train dataset
frames_root = "../output_frames/Train_Augmentation"
labels_csv = "../Labels/TrainLabels.csv" 
X_train_Augmentation, X_train_Augmentation = process_dataset(frames_root, labels_csv)

Total de carpetas de video encontradas: 248
Procesando 248 carpetas de video...


Procesando videos:  10%|█         | 25/248 [00:44<06:37,  1.78s/it]


KeyboardInterrupt: 

# 4. Data Selection

In [ ]:
# For each student, view the quantity of videos per engagement level in a dataframe.
df_train = pd.read_csv("../Labels/AllLabels.csv")
df_train['StudentID'] = df_train['ClipID'].astype(str).str[:6]
engagement_student = df_train.pivot_table(index='StudentID', columns='Engagement', aggfunc='size', fill_value=0)
engagement_student.columns = [f'Engagement_{col}' for col in engagement_student.columns]  
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
display(engagement_student.T)

print(engagement_student.T.shape)

StudentID,110001,110002,110004,110005,110006,110007,110008,110010,110011,110012,110013,110014,110015,110016,110017,110041,111003,181374,200048,200049,200050,200054,202614,205601,210051,210052,210053,210055,210056,210057,210058,210059,210060,210061,226051,240846,248510,290428,303830,310062,310063,310064,310066,310068,310069,310070,310071,310072,310073,310074,310075,310076,310077,310078,310079,310080,310081,310082,310083,334463,337021,342227,350361,400018,400022,400023,400030,400033,401835,410019,410020,410024,410025,410026,410027,410028,410029,410030,410032,411021,411031,414081,459999,500039,500043,500044,500067,500095,510009,510034,510035,510036,510037,510038,510040,510042,510045,510046,510047,522129,556463,567496,591292,769862,799402,826382,826412,882654,907001,928901,940328,987736
Engagement_0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,1,3,1,0,0,0,0,0,1,0,0,1,1,0,0,7,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,6,0,0,0,0,1,0,1,0,5,0,0,3,1,1,1,1,0,1,0,6,0,4,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,1
Engagement_1,0,3,2,6,3,0,0,0,2,1,0,3,6,3,7,2,7,2,0,1,5,0,3,1,3,1,2,6,2,10,4,1,2,2,3,2,0,2,5,1,0,8,1,4,1,3,0,1,0,2,13,7,3,4,2,0,1,7,0,5,0,3,2,3,4,1,10,1,2,10,7,9,21,12,3,7,16,8,7,23,7,6,2,8,0,3,8,7,1,7,7,0,0,0,7,0,1,2,2,1,14,4,1,3,13,1,24,1,2,5,1,3
Engagement_2,12,17,22,68,44,100,5,1,38,38,1,61,51,18,50,15,38,45,20,10,61,51,25,43,38,81,55,73,36,72,58,49,69,51,46,47,16,25,30,32,27,32,31,10,30,51,7,33,25,50,69,26,65,38,59,1,32,23,17,57,17,34,29,75,40,32,32,63,30,61,52,44,66,60,15,53,79,14,35,67,28,28,20,73,6,71,74,71,39,41,51,4,12,61,74,75,30,81,53,26,37,29,23,22,37,39,32,23,10,24,23,12
Engagement_3,102,21,10,19,32,42,4,0,35,48,6,25,20,10,19,11,20,64,11,3,43,38,49,43,49,29,69,41,47,36,30,74,49,70,46,40,64,66,47,53,43,13,13,5,7,29,2,79,21,55,25,10,46,74,26,2,13,19,12,46,73,60,81,40,15,21,15,44,51,15,45,14,24,12,11,13,23,7,26,18,18,77,60,27,7,45,44,34,44,32,25,2,19,63,31,54,57,39,28,82,43,45,59,65,55,39,27,67,43,55,43,25


(4, 112)


In [ ]:
# Logic to select the same quantity of videos per engagement level, with a maximum of 61 videos per engagement level.
root_dir = "../Daisee/DataSet/Train/"
max_per_engagement = 61
df_train['VideoFolderID'] = df_train['ClipID'].astype(str)
grouped = df_train.groupby('StudentID')
final_selection = []


global_engagement_counts = {0: 0, 1: 0, 2: 0, 3: 0}
used_engagement_by_student = {}
student_video_counts = {}

# Lista aleatoria de estudiantes válidos
students_list = [s for s in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, s)) and s in grouped.groups]
random.shuffle(students_list)

# Paso 1: seleccionar todos los de engagement 0
for student_id in students_list:
    student_videos = grouped.get_group(student_id)
    engagement_0 = student_videos[student_videos['Engagement'] == 0]

    for _, row in engagement_0.iterrows():
        final_selection.append({
            'StudentID': student_id,
            'VideoFolderID': row['VideoFolderID'],
            'Engagement': 0
        })
        global_engagement_counts[0] += 1
        used_engagement_by_student.setdefault(student_id, set()).add(0)
        student_video_counts[student_id] = student_video_counts.get(student_id, 0) + 1

# Paso 2: para engagement 1, luego 2, luego 3 — solo estudiantes aún no usados
for eng_level in [1, 2, 3]:
    for student_id in students_list:
        if global_engagement_counts[eng_level] >= max_per_engagement:
            break

        if student_video_counts.get(student_id, 0) > 0:
            continue  # Este estudiante ya tiene algún video seleccionado

        student_videos = grouped.get_group(student_id)
        eng_subset = student_videos[student_videos['Engagement'] == eng_level]

        if not eng_subset.empty:
            row = eng_subset.sample(1).iloc[0]
            final_selection.append({
                'StudentID': student_id,
                'VideoFolderID': row['VideoFolderID'],
                'Engagement': eng_level
            })
            global_engagement_counts[eng_level] += 1
            used_engagement_by_student.setdefault(student_id, set()).add(eng_level)
            student_video_counts[student_id] = 1

# Paso 3: completar faltantes (hasta 2 videos por estudiante)
for eng_level in [1, 2, 3]:
    if global_engagement_counts[eng_level] >= max_per_engagement:
        continue

    remaining_videos = df_train[df_train['Engagement'] == eng_level].sample(frac=1, random_state=42)

    for _, row in remaining_videos.iterrows():
        if global_engagement_counts[eng_level] >= max_per_engagement:
            break

        student_id = row['StudentID']
        if student_video_counts.get(student_id, 0) >= 2:
            continue  # Ya tiene 2 videos

        if eng_level in used_engagement_by_student.get(student_id, set()):
            continue  # Ya se usó este nivel en este estudiante

        final_selection.append({
            'StudentID': student_id,
            'VideoFolderID': row['VideoFolderID'],
            'Engagement': eng_level
        })
        global_engagement_counts[eng_level] += 1
        used_engagement_by_student.setdefault(student_id, set()).add(eng_level)
        student_video_counts[student_id] = student_video_counts.get(student_id, 0) + 1

# Paso 4: si sigue faltando, permitir hasta 3 videos por estudiante
for eng_level in [1, 2, 3]:
    if global_engagement_counts[eng_level] >= max_per_engagement:
        continue

    remaining_videos = df_train[df_train['Engagement'] == eng_level].sample(frac=1, random_state=99)

    for _, row in remaining_videos.iterrows():
        if global_engagement_counts[eng_level] >= max_per_engagement:
            break

        student_id = row['StudentID']
        if student_video_counts.get(student_id, 0) >= 3:
            continue

        if eng_level in used_engagement_by_student.get(student_id, set()):
            continue

        final_selection.append({
            'StudentID': student_id,
            'VideoFolderID': row['VideoFolderID'],
            'Engagement': eng_level
        })
        global_engagement_counts[eng_level] += 1
        used_engagement_by_student.setdefault(student_id, set()).add(eng_level)
        student_video_counts[student_id] = student_video_counts.get(student_id, 0) + 1

df_selected_videos = pd.DataFrame(final_selection)
print(df_selected_videos.head())
print(f"\nTotal estudiantes seleccionados: {df_selected_videos['StudentID'].nunique()}")
print(f"Cantidad total de videos seleccionados: {df_selected_videos.shape[0]}")
print("\nDistribución por engagement:")
print(df_selected_videos['Engagement'].value_counts())

  StudentID   VideoFolderID  Engagement
0    110015  1100152070.avi           0
1    410028  4100281067.avi           0
2    110017  1100171004.avi           0
3    110017  1100171008.avi           0
4    410026  4100262006.avi           0

Total estudiantes seleccionados: 112
Cantidad total de videos seleccionados: 244

Distribución por engagement:
Engagement
0    61
1    61
2    61
3    61
Name: count, dtype: int64


In [ ]:
# View distribution of engagement levels for each student to check if the selection is balanced
pivot_table = df_selected_videos.pivot_table(index='StudentID', columns='Engagement', aggfunc='size', fill_value=0)
pivot_table.columns = [f'Engagement_{col}' for col in pivot_table.columns]  
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
display(pivot_table.T)
print(pivot_table.T.shape)

StudentID,110001,110002,110004,110005,110006,110007,110008,110010,110011,110012,110013,110014,110015,110016,110017,110041,111003,181374,200048,200049,200050,200054,202614,205601,210051,210052,210053,210055,210056,210057,210058,210059,210060,210061,226051,240846,248510,290428,303830,310062,310063,310064,310066,310068,310069,310070,310071,310072,310073,310074,310075,310076,310077,310078,310079,310080,310081,310082,310083,334463,337021,342227,350361,400018,400022,400023,400030,400033,401835,410019,410020,410024,410025,410026,410027,410028,410029,410030,410032,411021,411031,414081,459999,500039,500043,500044,500067,500095,510009,510034,510035,510036,510037,510038,510040,510042,510045,510046,510047,522129,556463,567496,591292,769862,799402,826382,826412,882654,907001,928901,940328,987736
Engagement_0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,1,3,1,0,0,0,0,0,1,0,0,1,1,0,0,7,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,6,0,0,0,0,1,0,1,0,5,0,0,3,1,1,1,1,0,1,0,6,0,4,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,1,0,0,0,0,1
Engagement_1,0,1,1,1,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,1,1,0,1,0,1,1,0,0,1,1,0,1,0,1,1,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,0,1,1,1,1,0,1,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,1,0,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,0
Engagement_2,1,0,0,1,0,1,1,1,1,1,0,0,1,1,0,1,0,1,1,0,1,1,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,1,1,1,1,0,0,1,1,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,1,0,1,1,1,0,1,1,1,1,1,1,0,1,0,1,0,0,0,0,1,1,1,0,1,0,0,0
Engagement_3,1,1,1,0,1,1,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,1,1,1,0,1,1,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,1,1,0,0,0,1,0,1,1,1,1,1,1,0,1,0,1,1,1,1,0,0,0,1,0,1,1,1


(4, 112)


Se seleccionaron todos los videos con Engagement 0, ya que es el nivel con la menor cantidad de videos, asegurando que cada estudiante tenga al menos un video seleccionado. Luego, se seleccionaron videos aleatoriamente de los niveles de Engagement 1, 2 y 3, distribuyéndolos de forma equilibrada, y asegurando que cada estudiante tuviera un máximo de 3 videos seleccionados en total. Este proceso garantiza una distribución balanceada de los niveles de Engagement entre los estudiantes, sin exceder el límite máximo de 61 videos por nivel.